In [64]:
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import os
import os.path as osp

import numpy as np

In [44]:
def create_label_map(labels):
    label_set = set()
    for lt in labels:
        label_set.add(lt)

    label_map = {}
    count = 0
    for l in label_set:
        label_map[l] = count
        count += 1
        
    return label_map

In [189]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        
        # part 1 of vgg-16
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3)
        self.conv2 = nn.Conv2d(32, 32, kernel_size=3)
        self.conv2_drop = nn.Dropout2d(p=0.3)
        self.norm2a = nn.BatchNorm2d(32)
        self.norm2b = nn.BatchNorm2d(32)
        
        # part 2 of vgg-16
        self.conv3 = nn.Conv2d(32, 64, kernel_size=3)
        self.conv4 = nn.Conv2d(64, 64, kernel_size=3)
        self.conv4_drop = nn.Dropout2d(p=0.3)
        self.norm4a = nn.BatchNorm2d(64)
        self.norm4b = nn.BatchNorm2d(64)
        
        # part 3 of vgg-16
        self.conv5 = nn.Conv2d(64, 128, kernel_size=3)
        self.conv6 = nn.Conv2d(128, 128, kernel_size=3)
        self.conv7 = nn.Conv2d(128, 128, kernel_size=3)
        self.conv7_drop = nn.Dropout2d(p=0.3)
        self.norm7a = nn.BatchNorm2d(128)
        self.norm7b = nn.BatchNorm2d(128)
        
        # part 4 of vgg-16
        self.conv8 = nn.Conv2d(128, 256, kernel_size=3)
        self.conv9 = nn.Conv2d(256, 256, kernel_size=3)
        self.conv10 = nn.Conv2d(256, 256, kernel_size=3)
        self.conv10_drop = nn.Dropout2d(p=0.3)
        self.norm10a = nn.BatchNorm2d(256)
        self.norm10b = nn.BatchNorm2d(256)
        
        # # part 5 of vgg-16
        # self.conv11 = nn.Conv2d(20, 20, kernel_size=3)
        # self.conv12 = nn.Conv2d(20, 20, kernel_size=3)
        # self.conv13 = nn.Conv2d(20, 20, kernel_size=3)
        # self.conv13_drop = nn.Dropout2d(p=0.3)
        
        self.fc1 = nn.Linear(4096, 2048)
        self.fc2 = nn.Linear(2048, 1024)
        self.fc3 = nn.Linear(1024, 10)

    def forward(self, x):
        
        x = F.relu(self.conv1(x))
        x = self.conv2_drop(self.conv2(x))
        x = self.norm2a(x)
        x = F.max_pool2d(x, 2)
        x = self.norm2b(x)
        
        x = F.relu(self.conv3(x))
        x = F.relu(self.conv4(x))
        x = self.norm4a(x)
        x = F.max_pool2d(x, 2)
        x = self.norm4b(x)
        
        x = F.relu(self.conv5(x))
        x = F.relu(self.conv6(x))
        x = F.relu(self.conv7(x))
        x = self.norm7a(x)
        x = F.max_pool2d(x, 2)
        x = self.norm7b(x)
        
        x = F.relu(self.conv8(x))
        x = F.relu(self.conv9(x))
        x = F.relu(self.conv10(x))
        x = self.norm10a(x)
        x = F.max_pool2d(x, 2)
        x = self.norm10b(x)
        
        # x = F.relu(self.conv11(x))
        # x = F.relu(self.conv12(x))
        # x = F.relu(F.max_pool2d(self.conv13_drop(self.conv13(x)), 2))
        
        # print(x.shape)
        x = x.view(-1, 4096)
        
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        
        x = F.relu(self.fc2(x))
        x = F.dropout(x, training=self.training)
        
        x = self.fc3(x)
        return x

In [167]:
def train(
        epoch,
        model,
        loss_func,
        train_loader,
        optimizer,
        log_interval,
        save_path
):  
    model.train()

    for batch_idx, (data, target) in enumerate(train_loader):

        optimizer.zero_grad()
        output = model(data)

        loss = loss_func(output, target)
        loss.backward()
        optimizer.step()

        if batch_idx % log_interval == 0:

            trained_count = batch_idx * len(data)
            total_count = len(train_loader.dataset)
            batch_percent = int(100 * batch_idx / len(train_loader))
            loss_val = loss.item() / len(data)
            print(
                f'Train Epoch: {epoch} ' + 
                f'[{trained_count}/{total_count} ({batch_percent}%)]' + 
                f'\tLoss: {loss_val:.6f}'
            )

    torch.save(model.state_dict(), osp.join(save_path, f"model_{epoch}.pt"))
    torch.save(optimizer.state_dict(), osp.join(save_path, f"opt_{epoch}.pt"))
    
    return loss.item() / len(data)


def test(
    model,
    loss_func,
    test_loader,
    test_type="Validation"
):
    model.eval()
    test_loss = 0
    correct = 0
    
    with torch.no_grad():
        for data, target in test_loader:
            output = model(data)
            test_loss += loss_func(output, target).item()
            pred = output.data.max(1, keepdim=True)[1]
            correct += pred.eq(target.data.view_as(pred)).sum()
            
        test_loss /= len(test_loader.dataset)
        
        print(
            f'{test_type} Average loss: {test_loss:.4f}, ' +
            f'Accuracy: {correct}/{len(test_loader.dataset)} ' + 
            f'({100.*correct/len(test_loader.dataset):.0f}%)'
        )
    
    return test_loss

In [150]:
n_epochs = 10
batch_size_train = 64
batch_size_valid = 1000
batch_size_test = 1000
learning_rate = 0.01
log_interval = 10
save_path = "C:/Users/aphri/Documents/t0002/pycharm/data/yg_ar/cnn/res0001"

if not osp.exists(save_path):
    os.makedirs(save_path)

random_seed = 1
torch.backends.cudnn.enabled = False
torch.manual_seed(random_seed)

In [151]:
from nebula.data.yg_ar.setup_data_image_hard import read_data
from nebula.common import to_scale_one, write_pickle, read_pickle

In [138]:
df_path = "C:/Users/aphri/Documents/t0002/pycharm/data/yg_ar/image_hard_df.pkl"
random_seed = 1
df, train_df, test_df, valid_df = read_data(df_path, random_seed)

In [139]:
label_map_a = create_label_map(df["label_a"])
label_map_at = create_label_map(df["label_at"])

In [164]:
train_x = train_df["image"].apply(lambda x: np.array([x.astype(np.float32)/225.0])).to_list()
train_y_a = train_df["label_a"].map(label_map_a).to_list()
train_y_at = train_df["label_at"].map(label_map_at).to_list()
train_loader_a = torch.utils.data.DataLoader(tuple(zip(train_x, train_y_a)), batch_size=batch_size_train, shuffle=True)
train_loader_at = torch.utils.data.DataLoader(tuple(zip(train_x, train_y_at)), batch_size=batch_size_train, shuffle=True)

In [163]:
valid_x = valid_df["image"].apply(lambda x:  np.array([x.astype(np.float32)/225.0])).to_list()
valid_y_a = valid_df["label_a"].map(label_map_a).to_list()
valid_y_at = valid_df["label_at"].map(label_map_at).to_list()
valid_loader_a = torch.utils.data.DataLoader(tuple(zip(valid_x, valid_y_a)), batch_size=batch_size_valid, shuffle=True)
valid_loader_at = torch.utils.data.DataLoader(tuple(zip(valid_x, valid_y_at)), batch_size=batch_size_valid, shuffle=True)

In [161]:
test_x = test_df["image"].apply(lambda x:  np.array([x.astype(np.float32)/225.0])).to_list()
test_y_a = test_df["label_a"].map(label_map_a).to_list()
test_y_at = test_df["label_at"].map(label_map_at).to_list()
test_loader_a = torch.utils.data.DataLoader(tuple(zip(test_x, test_y_a)), batch_size=batch_size_test, shuffle=True)
test_loader_at = torch.utils.data.DataLoader(tuple(zip(test_x, test_y_at)), batch_size=batch_size_test, shuffle=True)

In [ ]:
model = Net()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
loss_func = nn.CrossEntropyLoss(size_average=False)

train_res = []
test_res = []

test(
    model=model,
    loss_func=loss_func,
    test_loader=valid_loader_a
)

for epoch in range(1, n_epochs + 1):
    train_loss = train(
        epoch=epoch,
        model=model,
        loss_func=loss_func,
        train_loader=train_loader_a,
        optimizer=optimizer,
        log_interval=log_interval,
        save_path=save_path
    )
    train_res.append((epoch, train_loss))

    test_loss = test(
        model=model,
        loss_func=loss_func,
        test_loader=valid_loader_a
    )
    test_res.append((epoch, test_loss))

c:\users\aphri\miniconda3\lib\site-packages\torch\nn\_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))


Validation Average loss: 2.3028, Accuracy: 176/1760 (10%)
Train Epoch: 1 [0/16080 (0%)]	Loss: 2.306470
Train Epoch: 1 [640/16080 (3%)]	Loss: 3.753809
Train Epoch: 1 [1280/16080 (7%)]	Loss: 2.838478
Train Epoch: 1 [1920/16080 (11%)]	Loss: 2.742681
Train Epoch: 1 [2560/16080 (15%)]	Loss: 3.123515
Train Epoch: 1 [3200/16080 (19%)]	Loss: 2.524794
Train Epoch: 1 [3840/16080 (23%)]	Loss: 2.635178
Train Epoch: 1 [4480/16080 (27%)]	Loss: 2.368749
Train Epoch: 1 [5120/16080 (31%)]	Loss: 2.317142
Train Epoch: 1 [5760/16080 (35%)]	Loss: 2.366688
Train Epoch: 1 [6400/16080 (39%)]	Loss: 2.383743
Train Epoch: 1 [7040/16080 (43%)]	Loss: 2.301859
Train Epoch: 1 [7680/16080 (47%)]	Loss: 2.303848
Train Epoch: 1 [8320/16080 (51%)]	Loss: 2.410814
Train Epoch: 1 [8960/16080 (55%)]	Loss: 2.304687
Train Epoch: 1 [9600/16080 (59%)]	Loss: 2.313536
Train Epoch: 1 [10240/16080 (63%)]	Loss: 2.303791
Train Epoch: 1 [10880/16080 (67%)]	Loss: 2.303215
Train Epoch: 1 [11520/16080 (71%)]	Loss: 2.319545
